In [25]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline
from sentence_transformers import SentenceTransformer



# Load Serbian embeddings
# embedding_model = SentenceTransformer('djovak/embedic-large')
embedding_model = HuggingFaceEmbeddings(model_name="djovak/embedic-large")
#HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

# Load law book corpus (assuming it's a text file)
loader = TextLoader("data/krivicni_zakonik.txt", encoding="utf-8")
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Create FAISS vector store
vectorstore = FAISS.from_documents(texts, embedding_model)

# Load the LLM (Mistral or Llama)
# llm_pipeline = pipeline("text-generation", model="mistralai/Mistral-7B", device=0)
llm_pipeline = pipeline('text-generation', model='jerteh/gpt2-orao', max_new_tokens=600, device=-1)
llm = HuggingFacePipeline(pipeline=llm_pipeline)


# Create RAG pipeline
# TODO: Investigate alternate solutions for RAG!
qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever(), return_source_documents=True)

# # Ask a question
# # query = "Koji su osnovni principi krivičnog prava u Srbiji?"
query = "Na kojoj teritoriji važi krivični zakon Srbije?"
# query = "U kom članu se definiše teritorija važenja krivičnog zakona Srbije?"

result = qa(query)

# Print answer with sources
print("Odgovor:", result["result"])
for doc in result["source_documents"]:
    print("\nIzvor:", doc.metadata["source"])
    print(doc.page_content)


model.safetensors:   9%|8         | 273M/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.17M [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Odgovor: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

(3) Na učinioca krivičnog dela koje je predviđeno zakonom sa određenim vremenskim trajanjem primenjuje se taj zakon, bez
obzira na to kad mu se sudi, ako tim zakonom nije drukčije određeno.
Važenje krivičnog zakonodavstva na teritoriji Srbije
Član 6
(1) Krivično zakonodavstvo Republike Srbije važi za svakog ko na njenoj teritoriji učini krivično delo.
(2) Krivično zakonodavstvo Srbije važi i za svakog ko učini krivično delo na domaćem brodu, bez obzira gde se brod nalazi u
vreme izvršenja dela.

vreme izvršenja dela.
(3) Krivično zakonodavstvo Srbije važi i za svakog ko učini krivično delo u domaćem civilnom vazduhoplovu dok je u letu ili u
domaćem vojnom vazduhoplovu, bez obzira gde se vazduhoplov nalazio u vreme izvršenja krivičnog dela.
(4) Ako je u slučajevima iz st. 1. do 3. ovog člana u stranoj državi pokrenut

In [26]:
result.keys()

dict_keys(['query', 'result', 'source_documents'])

In [27]:
# query = "U kom članu se definiše teritorija važenja krivičnog zakona Srbije?"
# query = "Šta se smatra pomaganjem u izvršenju krivičnog dela?"
query = "Da li će se kazniti počinilac ako učini krivično delo po naređenju pretpostavljenog u vojsci?"
result = qa(query)

# Print answer with sources
print("Odgovor:", result["result"])
# for doc in result["source_documents"]:
#     print("\nIzvor:", doc.metadata["source"])
#     print(doc.page_content)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Odgovor: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

vojnom licu izreći disciplinska kazna, odnosno mera utvrđena zakonom, ako je delo dobilo naročito lak vid i ako to zahtevaju
interesi službe.
Krivično delo izvršeno po naređenju pretpostavljenog
Član 430
Neće se kazniti potčinjeni ako učini krivično delo po naređenju pretpostavljenog koje se tiče službene dužnosti, osim ako se
naređenje odnosilo na izvršenje krivičnog dela za koje se može izreći pet godina zatvora ili teža kazna, a potčinjeni je znao
da izvršenje naređenja predstavlja krivično delo.

učinilac će se kazniti za delo iz stava 1. novčanom kaznom ili zatvorom do jedne godine, a za delo iz stava 2. zatvorom od tri
meseca do tri godine.
Protivljenje pretpostavljenom
Član 401
(1) Vojno lice koje se zajedno sa drugim vojnim licima usprotivi naređenju pretpostavljenog u vezi sa službom i neće da ga
izvrši ili

In [30]:
len(result["source_documents"])

4

In [28]:
result["source_documents"][0]

Document(id='bca9cb8f-90fc-45f5-8b76-9c58b16f0697', metadata={'source': 'data/krivicni_zakonik.txt'}, page_content='vojnom licu izreći disciplinska kazna, odnosno mera utvrđena zakonom, ako je delo dobilo naročito lak vid i ako to zahtevaju\ninteresi službe.\nKrivično delo izvršeno po naređenju pretpostavljenog\nČlan 430\nNeće se kazniti potčinjeni ako učini krivično delo po naređenju pretpostavljenog koje se tiče službene dužnosti, osim ako se\nnaređenje odnosilo na izvršenje krivičnog dela za koje se može izreći pet godina zatvora ili teža kazna, a potčinjeni je znao\nda izvršenje naređenja predstavlja krivično delo.')

In [29]:
result["source_documents"][1]

Document(id='25b19619-0232-4d70-8e11-03dd6801eae3', metadata={'source': 'data/krivicni_zakonik.txt'}, page_content='učinilac će se kazniti za delo iz stava 1. novčanom kaznom ili zatvorom do jedne godine, a za delo iz stava 2. zatvorom od tri\nmeseca do tri godine.\nProtivljenje pretpostavljenom\nČlan 401\n(1) Vojno lice koje se zajedno sa drugim vojnim licima usprotivi naređenju pretpostavljenog u vezi sa službom i neće da ga\nizvrši ili odbije da vrši svoju dužnost,\nkazniće se zatvorom od tri meseca do tri godine.\n(2) Ako je delo iz stava 1. ovog člana izvršeno organizovano,')